## Lab-3: Logistic Regression and Gradient descent
-----

## Introduction 
##### In this lab, we will discuss Logistic Regression (LR). This is a very important problem in machine learning and is applied to solve a variety of real life tasks. For example, financial services such as banks can rely on historical data of customers' credit record for loan approval by humans to automate the decision making process. A logistic regression model can be trained on these data, and then used to predict loan acceptance for future applicants. In particalar in this lab, you will apply Logistic Regression to predict survival on Titanic and learn to analyze classification results. 

### Learning outcomes:
1. Compare classification problem (data) with regression problem (data)
2. Understand the core concepts of LR (sigmoid function and loss function)
3. Manual Feature Selection (are all the features relevant for the task?)
4. Perform data pre-processing and preparation. Which pre-precessing technique should be aplied and why? 
5. Implement Logistic Regression with sklearn
6. Interprete the results of logistic regression (accuracy, confusion matrix, recall, precision, f1-score)
7. Strength and weakness of logistic regression
8. LR with gradient descent 
-----

### Concepts Visualization

##### Here are the main equations behind Logistic Regression:


##### Given a data point $(x_i, y_i)$ where $x_i=\{x_i^1,x_i^2, x_i^3,...,x_i^n\}$;  $x_i^j$ represents the feature $j$ and $y_i$ the associated class label.

#### Considering the hypothesis function $f(x_i) = \beta_0 + \beta_1 x_i^1 + + \beta_2 x_i^2 + ... + \beta_n x_i^n$ the output the LR model is the sigmoid function applied to $f(x_i)$, i.e., 


##### $\hat p(x_i) = \frac{e^{f(x_i)}}{1+e^{f(x_i)}} = \frac{1}{1+e^{-f(x_i)}}$

#### Components of the loss function


##### Given the predicted $\hat{p}(x_i)$ of the $x_i$ The loss of classifying the data point  is computed as follow:
 
$L(\hat{p}(x_i), y_i) = \left\{\begin{matrix}
-\log (\hat{p}(x_i)) && y_i = 1\\ 
-\log (1 -\hat{p}(x_i)) && y_i = 0 
\end{matrix}\right.$

$L(\hat{p}(x_i), y_i) = -y_i\log (\hat{p}(x_i)) - (1 - y)_i\log (1 -\hat{p}(x_i))$
 

#### The goal of the LR model is to find the values of the parameters $\{\beta_i\}_{i=0}^{n}$ such that the average loss $L$ on the training dataset is minimized

$\hat y = \left\{\begin{matrix}
1 && \hat p(x) > threshold\\ 
0 && otherwise 
\end{matrix}\right.$

 
### Open Questions
1. What problem does Logistic Regression solve?  
2. What is the output of Logistic Regression model? 
3. Why can't we use average MSE as a loss function (as we did in Linear Regression)?   
 
Let's now see how the shape of $\hat p(x)$ depend on its parameters for a single feature: $\beta_0 + \beta_1 x_i$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
x = np.arange(-10, 10, 0.01)

def plot(b0, b1):
    p = 1 / (1 + np.exp(-(b0 + b1 * x)))
    label = "b0 = {}, b1 = {}".format(b0, b1)
    plt.plot(x, p, label=label)

plot(0,1)
plot(0,2)
plot(0,3)
plot(5,1)
plt.legend()
plt.grid()
plt.title('Sigmoid function')
plt.show()

What is controlled by parameters b0, b1?  
<br><br> 
Let's now visualize the components of the Loss function:

In [ ]:
x = np.arange(0.001, 1, 0.001)
y1 = -np.log(x)
y0 = -np.log(1-x)

plt.plot(x,y1, label="if y = 1")
plt.plot(x,y0, label="if y = 0")
plt.legend()
plt.show()

What are the benefits of using logarithms in the loss function?   <br><br>

### Loading Data

In [ ]:
import pandas as pd
data = pd.read_csv('titanic.csv')
data

In [ ]:
# removing name column
data = data.drop(['name'], axis = 1)

What do we know about the output class distribution? What percentage of people had survived? Is it important to have the same distribution in test and train sets? Why?


#### Task 1 .

Find and print the proporion of the positive (survived) class in the dataset, then split data to train and test sets preserving that proportion. Finally, as a check, calculate and print proportions in resulting sets.

In [ ]:
from sklearn.model_selection import train_test_split

# find and print the proportion of positive samples in data
print('% of positive samples in whole data:', 0)

# split data DataFrame into label column ("survived") and the rest columns as features
data_label = None
data_feature = None

# split by train_test_split
x_train, x_test, y_train, y_test = None

# find and print the proportion of positive samples in train and test sets, make sure they are approx same
print('% of positive samples in train set:', 0)
print('% of positive samples in test set:', 0)

#### Data preprocessing

... is done for you! But you are free to change it if needed.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# imputing missing values
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(x_train)
x_train = pd.DataFrame(imputer.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_test.columns)

# one-hot-encode categorical features
def ohe_new_features(df, features_name, encoder):
    new_feats = encoder.transform(df[features_name])
    # create dataframe from encoded features with named columns
    new_cols = pd.DataFrame(new_feats, columns=encoder.get_feature_names_out(features_name))
    new_df = pd.concat([df, new_cols], axis=1)    
    new_df.drop(features_name, axis=1, inplace=True)
    return new_df

encoder = OneHotEncoder(sparse=False, drop='first')
f_names = ['sex', 'embarked']
encoder.fit(x_train[f_names])
x_train = ohe_new_features(x_train, f_names, encoder)
x_test = ohe_new_features(x_test, f_names, encoder)

# feature scaling
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
x_train

### Logistic Regression

Now we are ready to see Logistic Regression in practice. 

#### Task 2. 
Fit Logistic Regression and output Accuracy, Precision,  Recall and F1 scores on test set.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# fit Logistic Regression
clf = None

# caclulate and print metrics
print(f'Testing accuracy = {metrics.accuracy_score(y_test, y_test_pred)}')
print(f'Testing precision = {metrics.precision_score(y_test, y_test_pred)}')
print(f'Testing recall = {metrics.recall_score(y_test, y_test_pred)}')
print(f'Testing F1-score = {metrics.f1_score(y_test, y_test_pred)}')

#### Question

Based on the metrics, is our model good enough? 

What about Logistic Regression Coefficients? Can we interpret them?

In [ ]:
print(*[a for a in zip(list(x_train.columns), clf.coef_[0])], sep='\n')

How can we choose a threshold? Let's try different ones and see their effect on metrics.

#### Task 3. 

Calculate Accuracy, Precision, and Recall values for each of the given threshold values and plot them.

In [ ]:
# Thresholds for LR 
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Probabilities of each sample from x_test
# Hint: LR has a function predict_proba
pred_proba = ???

# Accuracy, precision, recall
results = [[],[],[]]
for i in thresholds:
    # Write your code here: if the element from pred_proba > threshold,
    # add 1 to y_test_pred_thr, otherwise add 0
    y_test_pred_thr = ???
    # Compare y_test_pred_thr with y_test by metrics and add them to results
    pass
    
plt.plot(thresholds, results[0], label = 'accuracy')   
plt.plot(thresholds, results[1], label = 'precision')   
plt.plot(thresholds, results[2], label = 'recall')
plt.title('Threshold Selection')
plt.xlabel('threshold')
plt.ylabel('score')
plt.legend()
plt.grid()
plt.show()

### Confusion matrix

1. Why one may need confusion matrix? 
2. What new information does it show? 

<table><tr><td><img src='https://rasbt.github.io/mlxtend/user_guide/evaluate/confusion_matrix_files/confusion_matrix_1.png'></td></tr></table>

#### In multiclass (more than 2) case: 
<img align='center' src='https://www.researchgate.net/profile/Emanuele_Principi/publication/324226324/figure/fig2/AS:612091408941056@1522945372985/Normalized-confusion-matrix-of-best-performing-models-on-devel-subset-a-SVM.png' style='width: 350px;'>

where V, O, T, E - names of the classes

In [ ]:
from sklearn.metrics import confusion_matrix

titanic_true_labels = np.array(y_test)
titanic_pred_labels = y_test_pred

print(f'Confusion Matrix for Titanic:  \n {confusion_matrix(titanic_true_labels, titanic_pred_labels)}')

In seaborn this matrix is:

In [ ]:
import seaborn as sns

ax= plt.subplot()
sns.heatmap(confusion_matrix(titanic_true_labels, titanic_pred_labels), annot=True, fmt='g', ax=ax)  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Not Survived', 'Survived']); ax.yaxis.set_ticklabels(['Not Survived', 'Survived'])

#### Question

Which class (survived, not survived) is more precisely predicted?

### <center> Gradient Descent: Losgistic regression with Numpy </center>

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from IPython.display import clear_output

### Generate dataset

In [ ]:
data, labels = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=0)

### Visualize the data

In [ ]:
colors = {0:"red", 1:'blue'}
label_values = [0, 1]

for i in label_values:
    idx = np.where(labels == i)
    plt.scatter(data[idx, 0], data[idx, 1], c=colors[i])
plt.show()

### Define the Logistic Sigmoid Function $\sigma(z)$

$$ \sigma(z) = \frac{1}{1+e^{-z}}$$

In [ ]:
def sigmoid(x): 
    # return the sigmoid   
    return 1/ (1 + np.exp(-x))

### Compute the Cost Function $J(\theta)$ and Gradient

The objective of logistic regression is to minimize the cost function

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} [ y^{(i)}log(h_{\theta}(x^{(i)})) + (1 - y^{(i)})log(1 - (h_{\theta}(x^{(i)}))]$$

where the gradient of the cost function is given by

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})x_j^{(i)}$$

In vector form:
$$ \frac{\partial J(\theta)}{\partial \theta} = \frac{1}{m} \textbf{X}^T * (h_{\theta}(\textbf{X}) - \textbf{y}) $$

where **X** - m by j input matrix, **y** - m by 1 target vector, * - [dot multiplication](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)

#### Task 4. 

Complete the code to compute the predicted label, error, and gradient

In [ ]:
# return the total error and the gradient
def compute_cost(theta, x, y):
    m = len(y)
    # aka h_theta(x^i)
    y_pred = None 
    # The part of J(theta) under the sum
    error = None 
    # J(theta)
    cost = -1 / m * sum(error)
    # Compute in a vector form 
    gradient = None 
    return cost[0] , gradient

In [ ]:
### Plot the data with decision boundary
def plot_data_with_decision(data, labels, theta):
    colors = {0:"red", 1:'blue'}
    label_values = [0, 1]

    x_boundary = np.array([np.min(data[:, 1]), np.max(data[:, 1])])
    y_boundary = -(theta[0] + theta[1] * x_boundary) / theta[2]

    for i in label_values:
        idx = np.where(labels == i)
        plt.scatter(data[idx, 0], data[idx, 1], c=colors[i])
    plt.plot(x_boundary, y_boundary)
    plt.xlim([-2, 6])
    plt.show()

### Initialize the parameters of the hypothesis 

In [ ]:
n_feature = data.shape[1]
n_samples = data.shape[0]

X = np.append(np.ones((n_samples, 1)), data, axis=1) #include intercept
y = labels.reshape(n_samples, 1)

theta = np.random.randn(n_feature+1).reshape(n_feature+1, 1) # initialize the weights (parameters)
 
print(f"Initial values of theta : {theta}")
cost, gradient = compute_cost(theta, X, y)
print(f"Initial gradient : {gradient}\nInitial cost {cost}")

plot_data_with_decision(data,labels, theta)

### Apply gradient descent to minimize the loss function

Minimize the cost function $J(\theta)$ by updating the below equation and repeat until convergence
$\theta_j := \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}$ (simultaneously update $\theta_j$ for all $j$)

In [ ]:
costs = []
iterations = 300
lr = 0.01 # the learning rate
for i in range(iterations):  
    cost, gradient = compute_cost(theta, X, y)
    theta -= (lr * gradient) # simultaneously update the parameters with thier respective gradients
    costs.append(cost)    

### Plotting the Convergence of $J(\theta)$
Plot $J(\theta)$ against the number of iterations of gradient descent:

In [ ]:
plt.plot(costs)
plt.xlabel("Iterations")
plt.ylabel("$J(\Theta)$")
plt.title("Values of Cost Function over iterations of Gradient Descent")

### Plot the final decision boundary

In [ ]:
colors = {0:"red", 1:'blue'}
label_values = [0, 1]

x_boundary = np.array([np.min(X[:, 1]), np.max(X[:, 1])])
y_boundary = -(theta[0] + theta[1] * x_boundary) / theta[2]

for i in label_values:
    idx = np.where(labels == i)
    plt.scatter(data[idx, 0], data[idx, 1], c=colors[i])
plt.plot(x_boundary, y_boundary)
plt.show()

In [ ]:
def predict(theta, X):
    results = X.dot(theta)
    return results > 0.5

print(f"Accuracy: {metrics.accuracy_score(predict(theta, X), y)}")

### Further reading

Classification for more than two classes may be solved by One-vs-Rest or One-vs-One strategies. [link](https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/)